In [1]:
import os

In [2]:
os.getcwd()

'd:\\iNeuron\\Complete Project\\Deep-CNN_Classifier\\research'

In [3]:
os.chdir('../')

In [4]:
os.getcwd()

'd:\\iNeuron\\Complete Project\\Deep-CNN_Classifier'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir : Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir : Path
    trained_model_path: Path
    updated_base_model_path : Path
    training_data: Path
    params_epoch: int
    params_batch_size: int
    params_image_size: list
    params_is_augmentation: bool

In [6]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories

In [7]:
from pathlib import Path 

class ConfigurationManager:
    def __init__(self, 
                config_file_path = CONFIG_FILE_PATH, 
                params_file_path = PARAMS_FILE_PATH):
        try:
            self.config = read_yaml(config_file_path)
            self.params = read_yaml(params_file_path)
            create_directories([self.config.artifacts_root])
        except Exception as e:
            raise e

    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        try:
            config = self.config.prepare_callbacks

            root_dir = config.root_dir
            model_checkpoint_dir = os.path.dirname(config.checkpoint_model_filepath)   

            create_directories([
                Path(model_checkpoint_dir),
                Path(config.tensorboard_root_log_dir)
            ])         

            prepare_callback_config = PrepareCallbacksConfig(
                root_dir=Path(root_dir),
                tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
                checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
            )
            return prepare_callback_config
        except Exception as e:
            raise e

    def get_training_config(self):
        try:
            training = self.config.training
            prepare_base_model = self.config.prepare_base_model
            data_ingestion = self.config.data_ingestion
            params=self.params

            create_directories([Path(training.root_dir)])

            training_config = TrainingConfig(
                root_dir = Path(training.root_dir),
                trained_model_path=Path(training.trained_model_path),
                updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
                training_data=Path(os.path.join(data_ingestion.unzip_dir,"PetImages")),
                params_epoch=params.EPOCHS,
                params_batch_size=params.BATCH_SIZE,
                params_image_size=params.IMAGE_SIZE,
                params_is_augmentation=params.AUGMENTATION
            )
            return training_config
        except Exception as e:
            raise e
        
        

In [8]:
import tensorflow as tf
import time 
import os

class PrepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    #Enable visualizations for TensorBoard.
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [9]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epoch,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [10]:
import scipy

In [11]:
try:
    config = ConfigurationManager()
    prepare_callbacks_model_config = config.get_prepare_callback_config()
    prepare_callbacks_model = PrepareCallbacks(config=prepare_callbacks_model_config)
    callback_list = prepare_callbacks_model.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training_model = Training(config=training_config)
    training_model.get_base_model()
    training_model.train_valid_generator()
    training_model.train(
        callback_list = callback_list
    )
except Exception as e:
    raise e 

Found 4998 images belonging to 2 classes.
Found 20000 images belonging to 2 classes.
 824/1250 [==================>...........] - ETA: 28:57 - loss: 7.5669 - accuracy: 0.6515

d:\iNeuron\Complete Project\Deep-CNN_Classifier\env\lib\site-packages\PIL\TiffImagePlugin.py:858: UserWarning: Truncated File Read
  warnings.warn(str(msg))


1250/1250 [==============================] - 5795s 5s/step - loss: 6.1889 - accuracy: 0.6906 - val_loss: 1.4464 - val_accuracy: 0.8700
